<a href="https://colab.research.google.com/github/costpetrides/Fluid-Dynamics-Navier-Stokes/blob/main/Hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp
import matplotlib.pyplot as plt

# --- Βοηθητικές συναρτήσεις ---

def normalize_vector(vec):
    norm = np.linalg.norm(vec)
    if norm == 0:
        raise ValueError("Input vector is zero vector!")
    return vec / norm

def projector_matrix(b_normalized):
    """ Δημιουργεί τον projector I - |b><b| """
    outer = np.outer(b_normalized, b_normalized)
    dim = len(b_normalized)
    identity = np.eye(dim)
    return identity - outer

def build_hamiltonian(A, b):
    """
    Κατασκευή Hermitian matrix H = A^T (I - |b><b|) A
    για χρήση στο VQE.
    """
    b_norm = normalize_vector(b)
    P = projector_matrix(b_norm)
    H = A.T @ P @ A
    return H

# --- Quantum Ansatz και QNode ---

def get_device(num_qubits):
    return qml.device("default.qubit", wires=num_qubits)

def ansatz(params, wires):
    """
    Strongly Entangling Layers Ansatz
    """
    qml.templates.StronglyEntanglingLayers(params, wires=wires)

def create_cost_function(H, dev, num_qubits, layers=3):
    """
    Δημιουργεί το cost function <ψ(θ)|H|ψ(θ)> για το VQE.

    Επιστρέφει:
    - cost_fn: συνάρτηση κόστους προς βελτιστοποίηση
    - init_params: αρχικές τυχαίες παράμετροι
    """

    num_params = qml.templates.StronglyEntanglingLayers.shape(n_layers=layers, n_wires=num_qubits)
    np.random.seed(42)
    init_params = np.random.normal(0, np.pi, size=num_params)

    @qml.qnode(dev)
    def circuit(params):
        ansatz(params, wires=range(num_qubits))
        return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

    def cost_fn(params):
        return circuit(params)

    return cost_fn, init_params

# --- VQE Optimization ---

def run_vqe(cost_fn, init_params, steps=200, lr=0.3, verbose=True):
    opt = qml.AdamOptimizer(stepsize=lr)
    params = init_params.copy()
    history = []

    for i in range(steps):
        params, cost = opt.step_and_cost(cost_fn, params)
        history.append(cost)
        if verbose and (i % 10 == 0 or i == steps-1):
            print(f"Step {i+1:03d}/{steps} | Cost = {cost:.8f}")
    return params, history

def plot_convergence(history):
    plt.figure(figsize=(8,5))
    plt.plot(history)
    plt.yscale("log")
    plt.xlabel("Optimization Step")
    plt.ylabel("Cost ⟨ψ|H|ψ⟩")
    plt.title("VQE Cost Function Convergence")
    plt.grid(True)
    plt.show()

# --- Quantum solver wrapper ---

def vqe_solver(A, b, layers=3, max_steps=200, lr=0.3, verbose=True):
    """
    Λύνει το σύστημα Ax=b χρησιμοποιώντας VQE (PennyLane simulator).

    1) Κάνει zero-padding σε δύναμη του 2
    2) Κατασκευάζει το Hamiltonian
    3) Βελτιστοποιεί το ansatz με VQE
    4) Επιστρέφει λύση (διανύσμα)

    Parameters:
    - A: np.ndarray, NxN matrix
    - b: np.ndarray, right-hand side vector (N,)
    - layers: int, βάθος ansatz
    - max_steps: int, βήματα optimizer
    - lr: float, learning rate
    - verbose: bool, εκτυπώσεις προόδου

    Returns:
    - solution: np.ndarray, λύση διανύσματος μεγέθους N
    """

    n = A.shape[0]
    n_qubits = int(np.ceil(np.log2(n)))
    dim = 2 ** n_qubits

    # zero-padding σε μέγεθος power of two
    A_pad = np.zeros((dim, dim))
    A_pad[:n, :n] = A
    b_pad = np.zeros(dim)
    b_pad[:n] = b

    H_mat = build_hamiltonian(A_pad, b_pad)
    H = qml.Hermitian(pnp.array(H_mat, dtype=np.complex128), wires=range(n_qubits))

    dev = get_device(n_qubits)
    cost_fn, init_params = create_cost_function(H_mat, dev, n_qubits, layers=layers)

    params_opt, history = run_vqe(cost_fn, init_params, steps=max_steps, lr=lr, verbose=verbose)

    # QNode για state
    @qml.qnode(dev)
    def get_state(params):
        ansatz(params, wires=range(n_qubits))
        return qml.state()

    qstate = get_state(params_opt)
    solution = qstate.real[:n]
    solution /= np.linalg.norm(solution) + 1e-12

    if verbose:
        plot_convergence(history)

    return solution

# --- Παραμετροποίηση και πλέγμα ---

class Grid:
    def __init__(self, Nx, Ny, Lx=1.0, Ly=1.0):
        self.Nx = Nx
        self.Ny = Ny
        self.Lx = Lx
        self.Ly = Ly

        self.dx = Lx / (Nx - 1)
        self.dy = Ly / (Ny - 1)

        # Κέντρα κυψελών (cell centers)
        self.x = np.linspace(0, Lx, Nx)
        self.y = np.linspace(0, Ly, Ny)

# --- Boundary Conditions για ταχύτητες (lid-driven cavity) ---

def apply_bc_velocity(u, v, Ulid=1.0):
    # u: velocity x-component, size (Ny, Nx+1)
    # v: velocity y-component, size (Ny+1, Nx)

    Ny, Nx1 = u.shape
    Ny1, Nx = v.shape

    # Οριζόντια ταχύτητα u
    # Lid (top): u = Ulid
    u[0, :] = Ulid
    # Bottom: u=0
    u[-1, :] = 0
    # Αριστερά και δεξιά τοίχοι: no-slip u=0
    u[:, 0] = 0
    u[:, -1] = 0

    # Κατακόρυφη ταχύτητα v
    # Όλα τα όρια v=0 (no-slip)
    v[0, :] = 0
    v[-1, :] = 0
    v[:, 0] = 0
    v[:, -1] = 0

# --- Διαφορικές πράξεις (πεδίο ταχυτήτων) ---

def gradient_x(phi, dx):
    return (phi[:, 1:] - phi[:, :-1]) / dx

def gradient_y(phi, dy):
    return (phi[1:, :] - phi[:-1, :]) / dy

def divergence(u, v, dx, dy):
    # u: (Ny, Nx+1), v: (Ny+1, Nx)
    du_dx = (u[:, 1:] - u[:, :-1]) / dx
    dv_dy = (v[1:, :] - v[:-1, :]) / dy
    return du_dx + dv_dy

# --- Pressure correction Poisson solver (Jacobi) ---

def pressure_correction_poisson(p, rhs, dx, dy, tol=1e-5, max_iter=1000):
    Ny, Nx = p.shape
    pn = p.copy()

    dx2 = dx * dx
    dy2 = dy * dy
    denom = 2 * (dx2 + dy2)

    for it in range(max_iter):
        p_old = pn.copy()

        pn[1:-1,1:-1] = ((dy2 * (p_old[1:-1, 2:] + p_old[1:-1, :-2]) +
                          dx2 * (p_old[2:, 1:-1] + p_old[:-2, 1:-1]) -
                          rhs[1:-1,1:-1]*dx2*dy2) / denom)

        # Boundary conditions (Neumann zero gradient)
        pn[:,0] = pn[:,1]
        pn[:,-1] = pn[:,-2]
        pn[0,:] = pn[1,:]
        pn[-1,:] = 0  # reference pressure at bottom wall

        error = np.linalg.norm(pn - p_old, ord=2)
        if error < tol:
            break

    return pn

# --- SIMPLE method iteration ---

def simple_iteration(u, v, p, rho, nu, dx, dy, dt, max_iter=100, tol=1e-5):
    Ny, Nx1 = u.shape
    Ny1, Nx = v.shape

    Nx = Nx1 - 1
    Ny = Ny1 - 1

    for iteration in range(max_iter):
        # Momentum equations for u and v (explicit Euler)

        un = u.copy()
        vn = v.copy()

        # u velocity update (internal points)
        for j in range(1, Ny-1):
            for i in range(1, Nx):
                u_xx = (u[j, i+1] - 2*u[j, i] + u[j, i-1]) / dx**2
                u_yy = (u[j+1, i] - 2*u[j, i] + u[j-1, i]) / dy**2

                u_conv = u[j, i]*(u[j, i] - u[j, i-1])/dx + \
                         v[j, i]*(u[j, i] - u[j-1, i])/dy

                pressure_grad = (p[j, i] - p[j, i-1]) / dx

                u[j, i] = un[j, i] + dt * (
                    nu * (u_xx + u_yy) - u_conv - (1/rho)*pressure_grad
                )

        # v velocity update (internal points)
        for j in range(1, Ny):
            for i in range(1, Nx-1):
                v_xx = (v[j, i+1] - 2*v[j, i] + v[j, i-1]) / dx**2
                v_yy = (v[j+1, i] - 2*v[j, i] + v[j-1, i]) / dy**2

                u_conv = u[j, i]*(v[j, i] - v[j, i-1])/dx + \
                         v[j, i]*(v[j, i] - v[j-1, i])/dy

                pressure_grad = (p[j, i] - p[j-1, i]) / dy

                v[j, i] = vn[j, i] + dt * (
                    nu * (v_xx + v_yy) - u_conv - (1/rho)*pressure_grad
                )

        apply_bc_velocity(u, v)

        # Calculate continuity residual (divergence)
        div = divergence(u, v, dx, dy)

        # Solve pressure correction
        p = pressure_correction_poisson(p, (rho/dt)*div, dx, dy)

        # Check convergence
        res = np.linalg.norm(div)
        if res < tol:
            print(f"SIMPLE converged in {iteration} iterations with residual {res:.3e}")
            break

    return u, v, p

# --- Αρχικοποίηση πεδίων ---

def initialize_fields(Nx, Ny):
    # Διαστάσεις πεδίων:
    # u: (Ny, Nx+1) λόγω staggered grid
    # v: (Ny+1, Nx)
    # p: (Ny, Nx)
    u = np.zeros((Ny, Nx+1))
    v = np.zeros((Ny+1, Nx))
    p = np.zeros((Ny, Nx))
    return u, v, p


import pennylane as qml
from pennylane import numpy as pnp

# --- Quantum device setup ---

n_qubits = 4  # Παράδειγμα, προσαρμόζεται ανάλογα με το πρόβλημα
dev = qml.device("default.qubit", wires=n_qubits)

# --- Hamiltonian ορισμός ---

def create_hamiltonian(coeffs, obs):
    # coeffs: λίστα αριθμών (συντελεστών)
    # obs: λίστα με qml.Pauli operators
    H = qml.Hamiltonian(coeffs, obs)
    return H

# --- Variational Ansatz ---

def ansatz(params, wires):
    # params σχήμα (layers, qubits, 3)
    layers = params.shape[0]
    for layer in range(layers):
        for i in range(len(wires)):
            qml.RX(params[layer, i, 0], wires=i)
            qml.RY(params[layer, i, 1], wires=i)
            qml.RZ(params[layer, i, 2], wires=i)
        # Ενδιάμεσο entanglement (πχ CNOT σε κυκλικό δίκτυο)
        for i in range(len(wires)):
            qml.CNOT(wires=[i, (i + 1) % len(wires)])

# --- VQE cost function ---

def vqe_cost(params, H):
    @qml.qnode(dev)
    def circuit():
        ansatz(params, wires=range(n_qubits))
        return qml.expval(H)
    return circuit()

# --- Optimization ---

def run_vqe(H, layers=2, max_iter=100, tol=1e-6):
    # Αρχικοποίηση παραμέτρων (layers x n_qubits x 3 rotations)
    params = pnp.random.uniform(0, 2 * np.pi, (layers, n_qubits, 3), requires_grad=True)

    opt = qml.AdamOptimizer(stepsize=0.1)
    energy = []
    prev = 0

    for it in range(max_iter):
        params, curr_energy = opt.step_and_cost(lambda v: vqe_cost(v, H), params)
        energy.append(curr_energy)

        if it > 0 and abs(curr_energy - prev) < tol:
            print(f"VQE converged at iteration {it} with energy {curr_energy:.6f}")
            break
        prev = curr_energy

    return params, energy[-1]

# --- Παράδειγμα Hamiltonian (2-qubit simple) ---

coeffs = [0.5, -0.7, 0.8]
obs = [qml.PauliZ(0), qml.PauliZ(1), qml.PauliX(0) @ qml.PauliX(1)]
H_example = create_hamiltonian(coeffs, obs)

# --- Τρέξιμο VQE ---

params_opt, E = run_vqe(H_example, layers=3, max_iter=200)
print("Optimized Energy:", E)

# --- Σύνδεση με κλασικό solver ---

def vqe_solve_pressure_system(Ap, rhs, verbose=False):
    """
    Συνδέει τον VQE με τον pressure correction solver:
    Δίνει λύση στο σύστημα Ap x = rhs μέσω ελαχιστοποίησης
    του H = Aᵀ (I - |b⟩⟨b|) A.

    Parameters:
        Ap    : (N, N) matrix από τον pressure solver
        rhs   : (N,) right-hand side vector
        verbose : αν True, εκτυπώνει κόστος σε κάθε iteration

    Returns:
        x_vqe : (N,) normalized solution vector
    """

    # 1. Zero padding to next power of 2
    n = Ap.shape[0]
    n_qubits = int(np.ceil(np.log2(n)))
    n_dim = 2 ** n_qubits

    Ap_padded = np.zeros((n_dim, n_dim))
    rhs_padded = np.zeros(n_dim)
    Ap_padded[:n, :n] = Ap
    rhs_padded[:n] = rhs

    # 2. Build projector: I - |b⟩⟨b|
    b_norm = rhs_padded / (np.linalg.norm(rhs_padded) + 1e-12)
    P = np.eye(n_dim) - np.outer(b_norm, b_norm)

    # 3. Hamiltonian: H = Aᵀ (I - |b⟩⟨b|) A
    H_matrix = Ap_padded.T @ P @ Ap_padded

    # 4. Create PennyLane Hamiltonian (Pauli decomposition)
    #    Χρησιμοποιούμε qml.Hermitian εδώ χωρίς να σπάσουμε σε PauliTerms
    H_qml = qml.Hermitian(pnp.array(H_matrix, dtype=np.complex128), wires=range(n_qubits))

    # 5. Run VQE
    def circuit(params):
        ansatz(params, wires=range(n_qubits))
        return qml.expval(H_qml)

    @qml.qnode(dev)
    def qnode(params):
        return circuit(params)

    def cost_fn(params):
        return qnode(params)

    depth = 3
    init_params = 0.01 * np.random.randn(depth, n_qubits, 3)
    opt = qml.AdamOptimizer(stepsize=0.3)

    params = init_params
    for step in range(150):  # ή 200
        params, cost = opt.step_and_cost(cost_fn, params)
        if verbose and step % 10 == 0:
            print(f"[VQE] Step {step:3d} | Cost = {cost:.6g}")

    # 6. Extract final quantum state and solution vector
    @qml.qnode(dev)
    def get_state(params):
        ansatz(params, wires=range(n_qubits))
        return qml.state()

    state = get_state(params)
    x_solution = state.real[:n]  # extract only first N values
    return x_solution / (np.linalg.norm(x_solution) + 1e-12)



# --- Παράμετροι πλέγματος και φυσικές σταθερές ---
nx = 8     # αριθμός κελιών x
ny = 8     # αριθμός κελιών y
Lx = 1.0
Ly = 1.0

dx = Lx / (nx - 1)
dy = Ly / (ny - 1)

rho = 1.0      # πυκνότητα
nu = 0.1       # κινηματική ιξώδες
dt = 0.01      # χρονικό βήμα
tolerance = 1e-4
max_iters = 200

# --- Πλέγμα και συντεταγμένες ---
grid = Grid(nx, ny)
X, Y = np.meshgrid(grid.x, grid.y)

# --- Αρχικοποίηση πεδίων ---
u, v, p = initialize_fields(nx, ny)

# --- Εφαρμογή οριακών συνθηκών ---
apply_bc_velocity(u, v)

# --- Κατασκευή πίνακα πίεσης Ap για σύστημα Ap x = rhs ---
def build_pressure_matrix(Nx, Ny, dx, dy):
    """
    Κατασκευάζει sparse πίνακα διακριτοποίησης Laplacian για εξίσωση πίεσης.
    Dirichlet/Neumann boundary conditions.
    """
    from scipy.sparse import lil_matrix

    N = (Nx - 2) * (Ny - 2)
    A = lil_matrix((N, N))

    dx2 = dx * dx
    dy2 = dy * dy
    denom = 2 * (dx2 + dy2)

    def idx(i, j):
        return i + (Nx - 2) * j

    for j in range(Ny - 2):
        for i in range(Nx - 2):
            row = idx(i, j)
            A[row, row] = -2 * (1/dx2 + 1/dy2)
            if i > 0:
                A[row, idx(i - 1, j)] = 1 / dx2
            if i < Nx - 3:
                A[row, idx(i + 1, j)] = 1 / dx2
            if j > 0:
                A[row, idx(i, j - 1)] = 1 / dy2
            if j < Ny - 3:
                A[row, idx(i, j + 1)] = 1 / dy2

    return A.toarray()

# --- Δημιουργία πίνακα Ap ---
Ap = build_pressure_matrix(nx, ny, dx, dy)


# ------------------------------
# 4. ΚΥΡΙΟΣ ΒΡΟΧΟΣ SIMPLE + VQE
# ------------------------------

for it in range(1, max_iters + 1):
    # 1. Predict velocities (u*, v*)
    u_star = u.copy()
    v_star = v.copy()

    # --- u* prediction ---
    # Χρήση σωστών slices για να ταιριάζουν τα shapes
    u_star[1:-1,1:-1] = u[1:-1,1:-1] - dt * (
        ((u[1:-1,1:-1] - u[1:-1,0:-2]) * u[1:-1,1:-1] +
         (v[1:-1, :-1] - v[0:-2, :-1]) * u[1:-1,1:-1]) / dx +
        (p[1:-1,1:] - p[1:-1,0:-1]) / dx -
        nu * (
            (u[1:-1,2:] - 2*u[1:-1,1:-1] + u[1:-1,0:-2]) / dx**2 +
            (u[2:,1:-1] - 2*u[1:-1,1:-1] + u[0:-2,1:-1]) / dy**2
        )
    )

    # --- v* prediction ---
    v_star[1:-1,1:-1] = v[1:-1,1:-1] - dt * (
        ((u[1:-1,1:-1] - u[1:-1,0:-2]) * v[1:-1,1:-1] +
         (v[1:-1,1:-1] - v[0:-2,1:-1]) * v[1:-1,1:-1]) / dy +
        (p[2:,1:-1] - p[1:-1,1:-1]) / dy -
        nu * (
            (v[1:-1,2:] - 2*v[1:-1,1:-1] + v[1:-1,0:-2]) / dx**2 +
            (v[2:,1:-1] - 2*v[1:-1,1:-1] + v[0:-2,1:-1]) / dy**2
        )
    )

    # Επαναφορά οριακών συνθηκών για u_star και v_star
    u_star[:, 0]  = 0.0
    u_star[:, -1] = 0.0
    u_star[0, :]  = 0.0
    u_star[-1, :] = 1.0  # lid-driven top wall

    v_star[:, 0]  = 0.0
    v_star[:, -1] = 0.0
    v_star[0, :]  = 0.0
    v_star[-1, :] = 0.0

    # 2. Pressure solve with quantum VQE
    rhs_p = ((u_star[1:-1,1:] - u_star[1:-1,0:-1]) / dx +
             (v_star[1:,1:-1] - v_star[0:-1,1:-1]) / dy) / dt

    rhsp = rhs_p.flatten()
    p_prime_flat = vqe_solve_pressure_system(Ap, rhsp, verbose=False)
    p_prime = p.copy()
    p_prime[1:-1,1:-1] = p_prime_flat.reshape((ny - 2, nx - 2))

    # 3. Velocity correction
    u[1:-1,1:-1] = u_star[1:-1,1:-1] - dt / dx * (p_prime[1:-1,1:-1] - p_prime[1:-1,0:-2])
    v[1:-1,1:-1] = v_star[1:-1,1:-1] - dt / dy * (p_prime[1:-1,1:-1] - p_prime[0:-2,1:-1])

    # Update pressure
    p = p_prime.copy()

    # Compute residual
    u_diff = np.linalg.norm(u - u_star)
    v_diff = np.linalg.norm(v - v_star)
    res = max(u_diff, v_diff)

    if it % 10 == 0 or it == 1:
        print(f"[{it:4d}] Residual: {res:.4e}")

    if res < tolerance:
        print(f"Converged at iteration {it}")
        break








# -------------------------------------
# Οπτικοποίηση τελικού πεδίου ταχυτήτων
# -------------------------------------

plt.figure(figsize=(6, 5))
plt.quiver(X[::2, ::2], Y[::2, ::2], u[::2, ::2], v[::2, ::2])
plt.title("Velocity field (Quantum–Classical VQE)")
plt.xlabel("x")
plt.ylabel("y")
plt.axis("equal")
plt.tight_layout()
plt.show()


VQE converged at iteration 77 with energy -1.441818
Optimized Energy: -1.441818338316557


ValueError: operands could not be broadcast together with shapes (7,7) (6,7) 